In [1]:
!ls
!ls data

data  PCA_generator.ipynb  spark-warehouse
app.py	data.csv


In [2]:
!pip install nltk

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from collections import Counter
import math
import re

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import *

sc = SparkContext('local')
sqlContext = SQLContext(sc)

ModuleNotFoundError: No module named 'findspark'

In [5]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from nltk import download
download('punkt')
download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ucsddse230/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ucsddse230/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
!head data/data.csv -n 5

In [7]:
tweets_df = sqlContext.read.csv('./data/data.csv',header=True, mode="DROPMALFORMED")

In [8]:
tweets_df.count(), tweets_df.show(5), tweets_df.createOrReplaceTempView("tweets")

+--------------------+-------------------+---------+--------+------------------+
|                Text|               Date|Favorites|Retweets|          Tweet ID|
+--------------------+-------------------+---------+--------+------------------+
|I have not heard ...|2016-12-21 13:29:38|    14755|    4055|811564284706689024|
|I would have done...|2016-12-21 13:24:29|    11129|    2789|811562990285848576|
|Campaigning to wi...|2016-12-21 13:15:14|    14906|    3925|811560662853939200|
|Yes, it is true -...|2016-12-20 20:27:57|    51424|   12578|811307169043849216|
|especially how to...|2016-12-20 13:09:18|    35699|    8008|811196778779463684|
+--------------------+-------------------+---------+--------+------------------+
only showing top 5 rows



(27703, None, None)

In [41]:
def safe_sent_tokenize(x):
    try:
        return sent_tokenize(x.Text)
    except TypeError:
        return [None]
    
tweet_sents = tweets_df.select('Text').rdd.flatMap(safe_sent_tokenize).filter(lambda x : not x == None)

In [42]:
tweet_sents.first()

'I have not heard any of the pundits or commentators discussing the fact that I spent FAR LESS MONEY on the win than Hillary on the loss!'

In [43]:
tokenized_tweet_sents = tweet_sents.map(lambda x : word_tokenize(x))
print(tokenized_tweet_sents.first())

['I', 'have', 'not', 'heard', 'any', 'of', 'the', 'pundits', 'or', 'commentators', 'discussing', 'the', 'fact', 'that', 'I', 'spent', 'FAR', 'LESS', 'MONEY', 'on', 'the', 'win', 'than', 'Hillary', 'on', 'the', 'loss', '!']


In [73]:
num_pattern = re.compile(r'\d+$')
year_pattern = re.compile(r'[12][90]\d\d$')
big_num = re.compile('\d[\d,]+$')

    
def apply_filters(x):
    if year_pattern.match(x):
        return '__year__'
    elif num_pattern.match(x):
        return '__num__'
    elif big_num.match(x):
        return '__bignum__'
    elif len(x) > 1 and '.' in x:
        return '__url__'
    else:
        return x
    
print([apply_filters(str(xi).lower()) for xi in ['1,000,000', '//facebook.com/hello', 'not', '10', 'any', 'of', '2019', 'pundits', 'or', 'commentators', 'discussing', 'the', 'fact', 'that', 'I', 'spent', 'FAR', 'LESS', 'MONEY', 'on', 'the', 'win', 'than', 'Hillary', 'on', 'the', 'loss', '!']])

['__bignum__', '__url__', 'not', '__num__', 'any', 'of', '__year__', 'pundits', 'or', 'commentators', 'discussing', 'the', 'fact', 'that', 'i', 'spent', 'far', 'less', 'money', 'on', 'the', 'win', 'than', 'hillary', 'on', 'the', 'loss', '!']


In [74]:
def lower_tag_filter(x):
    return pos_tag([apply_filters(str(xi).lower()) for xi in x])

In [75]:
pos_tagged_sents = tokenized_tweet_sents.map(lower_tag_filter)
print(pos_tagged_sents.first())

[('i', 'NNS'), ('have', 'VBP'), ('not', 'RB'), ('heard', 'VBN'), ('any', 'DT'), ('of', 'IN'), ('the', 'DT'), ('pundits', 'NNS'), ('or', 'CC'), ('commentators', 'NNS'), ('discussing', 'VBG'), ('the', 'DT'), ('fact', 'NN'), ('that', 'IN'), ('i', 'JJ'), ('spent', 'VBD'), ('far', 'RB'), ('less', 'JJR'), ('money', 'NN'), ('on', 'IN'), ('the', 'DT'), ('win', 'NN'), ('than', 'IN'), ('hillary', 'JJ'), ('on', 'IN'), ('the', 'DT'), ('loss', 'NN'), ('!', '.')]


In [76]:
pos_tagged_sents.cache()

PythonRDD[99] at RDD at PythonRDD.scala:48

In [77]:
word_freqs = pos_tagged_sents.flatMap(lambda x : x).map(lambda x : (x, 1)).groupByKey().mapValues(lambda x : len(x)).map(reversed).map(tuple).sortByKey(ascending = False).map(reversed).map(tuple)

In [78]:
print(word_freqs.take(10), word_freqs.count())

[(("''", "''"), 36886), (('.', '.'), 22547), (('``', '``'), 19289), (('@', 'JJ'), 18321), ((':', ':'), 15490), (('the', 'DT'), 15133), (('!', '.'), 14724), (('to', 'TO'), 9458), (('__url__', 'NN'), 7902), ((',', ','), 7885)] 38734


In [79]:
freqs_dict = Counter(dict(word_freqs.collect()))
len(freqs_dict), freqs_dict

(38734,
 Counter({("''", "''"): 36886,
          ('.', '.'): 22547,
          ('``', '``'): 19289,
          ('@', 'JJ'): 18321,
          (':', ':'): 15490,
          ('the', 'DT'): 15133,
          ('!', '.'): 14724,
          ('to', 'TO'): 9458,
          ('__url__', 'NN'): 7902,
          (',', ','): 7885,
          ('@', 'NNP'): 7597,
          ('a', 'DT'): 7596,
          ('is', 'VBZ'): 6894,
          ('you', 'PRP'): 6325,
          ('and', 'CC'): 6233,
          ('realdonaldtrump', 'NN'): 6155,
          ('in', 'IN'): 5962,
          ('of', 'IN'): 5683,
          ('#', '#'): 5293,
          ('for', 'IN'): 5163,
          ('on', 'IN'): 4477,
          ('i', 'NN'): 4212,
          ('@', 'NN'): 4013,
          ('http', 'NN'): 3864,
          ('be', 'VB'): 3762,
          ('will', 'MD'): 3373,
          ('it', 'PRP'): 3356,
          ('great', 'JJ'): 3338,
          ('trump', 'NN'): 2887,
          ('?', '.'): 2623,
          ('are', 'VBP'): 2459,
          ('at', 'IN'): 2426,
    

In [80]:
def get_context_pairs(wordlist, context_breadth = 3):
    if len(wordlist) == 1:
        return []
    context_pairs = []
    for i, word in enumerate(wordlist[1:]):
        for j in range(max(i - context_breadth + 1, 0), i + 1):
            context_pairs.append((word, wordlist[j]))
            context_pairs.append((wordlist[j], word))
    return context_pairs


pairs = get_context_pairs(word_tokenize('please hire me to work at your wonderful company.'))
print(pairs)

[('hire', 'please'), ('please', 'hire'), ('me', 'please'), ('please', 'me'), ('me', 'hire'), ('hire', 'me'), ('to', 'please'), ('please', 'to'), ('to', 'hire'), ('hire', 'to'), ('to', 'me'), ('me', 'to'), ('work', 'hire'), ('hire', 'work'), ('work', 'me'), ('me', 'work'), ('work', 'to'), ('to', 'work'), ('at', 'me'), ('me', 'at'), ('at', 'to'), ('to', 'at'), ('at', 'work'), ('work', 'at'), ('your', 'to'), ('to', 'your'), ('your', 'work'), ('work', 'your'), ('your', 'at'), ('at', 'your'), ('wonderful', 'work'), ('work', 'wonderful'), ('wonderful', 'at'), ('at', 'wonderful'), ('wonderful', 'your'), ('your', 'wonderful'), ('company', 'at'), ('at', 'company'), ('company', 'your'), ('your', 'company'), ('company', 'wonderful'), ('wonderful', 'company'), ('.', 'your'), ('your', '.'), ('.', 'wonderful'), ('wonderful', '.'), ('.', 'company'), ('company', '.')]


In [81]:
corr_pairs = pos_tagged_sents.flatMap(get_context_pairs).map(lambda x : (x ,1)).groupByKey().mapValues(len)
corr_pairs.first()

((('have', 'VBP'), ('i', 'NNS')), 173)

In [82]:
def calc_phi(x, freqs, sum_allwords):
    nw, nc = freqs[x[0][0]], freqs[x[0][1]]
    if nw == 0 or nc == 0:
        return 0
    return max(0, math.log((sum_allwords * x[1]) / nw * nc))

ex = ((('have', 'VBP'), ('i', 'NNS')), 173)
print(freqs_dict[ex[0][0]], freqs_dict[ex[0][1]], ex[1])
calc_phi(ex, freqs_dict, 100000)

1332 452 173


15.585462388199904

In [83]:
all_words = sum(freqs_dict.values())
all_words

614554

In [84]:
pairwise_correlations = corr_pairs.map(lambda x : (x[0], calc_phi(x, freqs_dict, all_words)))
pairwise_correlations.first()

((('have', 'VBP'), ('i', 'NNS')), 17.401189003679217)

In [85]:
word_ids = dict(zip(freqs_dict.keys(), np.arange(len(freqs_dict))))
word_ids

{("''", "''"): 0,
 ('.', '.'): 1,
 ('``', '``'): 2,
 ('@', 'JJ'): 3,
 (':', ':'): 4,
 ('the', 'DT'): 5,
 ('!', '.'): 6,
 ('to', 'TO'): 7,
 ('__url__', 'NN'): 8,
 (',', ','): 9,
 ('@', 'NNP'): 10,
 ('a', 'DT'): 11,
 ('is', 'VBZ'): 12,
 ('you', 'PRP'): 13,
 ('and', 'CC'): 14,
 ('realdonaldtrump', 'NN'): 15,
 ('in', 'IN'): 16,
 ('of', 'IN'): 17,
 ('#', '#'): 18,
 ('for', 'IN'): 19,
 ('on', 'IN'): 20,
 ('i', 'NN'): 21,
 ('@', 'NN'): 22,
 ('http', 'NN'): 23,
 ('be', 'VB'): 24,
 ('will', 'MD'): 25,
 ('it', 'PRP'): 26,
 ('great', 'JJ'): 27,
 ('trump', 'NN'): 28,
 ('?', '.'): 29,
 ('are', 'VBP'): 30,
 ('at', 'IN'): 31,
 ('he', 'PRP'): 32,
 ('&', 'CC'): 33,
 ('my', 'PRP$'): 34,
 ('with', 'IN'): 35,
 ('we', 'PRP'): 36,
 ('--', ':'): 37,
 ("n't", 'RB'): 38,
 ("'s", 'POS'): 39,
 ('thanks', 'NNS'): 40,
 ('-', ':'): 41,
 ('this', 'DT'): 42,
 ('your', 'PRP$'): 43,
 ('not', 'RB'): 44,
 ('…', 'NN'): 45,
 ('our', 'PRP$'): 46,
 ('__url__', 'JJ'): 47,
 ('that', 'IN'): 48,
 ('by', 'IN'): 49,
 ('was', 'VBD'

In [86]:
def convert_to_ids(x, word_ids):
    return (word_ids[x[0][0]], (word_ids[x[0][1]], x[1]))

convert_to_ids(((('have', 'VBP'), ('i', 'NNS')), 17.405604021888333), word_ids)

(55, (152, 17.405604021888333))

In [87]:
corellation_rdd = pairwise_correlations.map(lambda x : convert_to_ids(x, word_ids)).groupByKey().mapValues(dict).cache()
corellation_rdd.first()

(132,
 {0: 20.940074354663793,
  1: 20.447844298807553,
  2: 19.73216147670986,
  3: 20.82010871756523,
  4: 20.236739985123233,
  5: 20.965410800982223,
  6: 19.867570415092693,
  7: 19.83041765190577,
  8: 18.552060311005174,
  9: 19.702586149369363,
  10: 18.42568647454743,
  11: 19.978903280759443,
  12: 18.56974656693791,
  13: 16.943160092509668,
  14: 19.048771351922888,
  15: 19.11313939486872,
  16: 17.57720323371191,
  17: 17.529276500771015,
  18: 17.61233311115914,
  19: 18.126462224913485,
  20: 17.444901097542115,
  21: 16.13112254405965,
  22: 16.775871221632645,
  23: 16.450352974246357,
  24: 15.324988500106814,
  25: 15.215840325307488,
  26: 19.218120743065406,
  27: 17.507994687461665,
  28: 16.852015139136828,
  29: 16.756115939340766,
  31: 15.984893868417943,
  32: 19.233496563427515,
  33: 17.232145662864895,
  34: 16.619912279426458,
  35: 16.42218209727966,
  36: 15.435122294517297,
  37: 17.279406442524998,
  38: 20.14121458956428,
  39: 14.649233062258737,
 

In [88]:
corellation_rdd.count()

38733

In [89]:
len(freqs_dict), len(word_ids)

(38734, 38734)

In [90]:
corr_vectors = sc.parallelize([(idn, {}) for word, idn in word_ids.items()]).leftOuterJoin(corellation_rdd).mapValues(lambda x : x[1] if not x[1] == None else x[0]).sortByKey()
corr_vectors.take(5)

[(0,
  {32: 15.014903421204902,
   2: 22.511540631211755,
   132: 12.381638826585698,
   38: 15.236101871974459,
   288: 10.74399009869018,
   206: 12.407850839067063,
   0: 23.865271381135628,
   8: 20.01877946137636,
   3: 21.52120448773605,
   2027: 7.6493466331582685,
   4: 21.686637655125043,
   3506: 5.115649819200836,
   835: 9.003380132059938,
   6: 21.392859912963146,
   4891: 4.604824195434846,
   70: 14.42253597094957,
   1927: 7.002719468233217,
   1: 21.55932772559407,
   13: 18.847403697895977,
   14402: 2.813064726206791,
   14409: 2.813064726206791,
   18: 19.234967735538387,
   189: 14.363719407987027,
   54: 16.8249969653438,
   31: 15.26102416586831,
   28: 17.684785163454325,
   302: 10.903773442290788,
   320: 9.679998010668672,
   5: 19.448911782244185,
   1053: 8.72114766437572,
   1001: 8.902109601653637,
   25: 15.967060482660877,
   169: 12.7438269208778,
   85: 14.22362518449899,
   231: 10.900090196874492,
   4420: 5.452122055822049,
   63: 14.32735925358325

In [91]:
print('Uncorrelated words:')
for (uncorrid, _) in corr_vectors.filter(lambda x : len(x[1]) == 0).collect():
    for word, idn in word_ids.items():
        if idn == uncorrid:
            print(word)
            break
corr_vectors.count()

Uncorrelated words:
('courtmarshal', 'NN')


38734

In [92]:
length = corr_vectors.count()
rows = corr_vectors.map(lambda x : Vectors.sparse(length, x[1]))

mat = RowMatrix(rows)

In [93]:
svd = mat.computeSVD(200, computeU=True)

Py4JJavaError: An error occurred while calling o799.computeSVD.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1739.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1739.0 (TID 719, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 1338435 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1599)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1587)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1586)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1586)
	at org.apache.spar